# Retinopathy Classification

## Submit result to kaggle as Part3 HW: https://www.kaggle.com/c/retinopathy-classification-sai

## HINT

1.   Select model from keras application
2.   Transfer learning
3.   Data Augmentation
4.   More imaage preprocess (ex: histogram equlization, crop image, ... etc)



In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, utils, activations, losses, optimizers, metrics
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# Define Parameters
num_classes = 3
IMG_SIZE = 224

In [1]:
# download from google drive or upload file by yourself
!gdown --id '1ZrB69wSgzXLu8UZdPJb8hnB6K4eltcpu' --output retina-dataset.zip

Permission denied: https://drive.google.com/uc?id=1ZrB69wSgzXLu8UZdPJb8hnB6K4eltcpu
Maybe you need to change permission over 'Anyone with the link'?


In [ ]:
# unzip dataset file
!unzip -q retina-dataset.zip

#### Data Visualization

In [ ]:
paths = glob('retina-train/*.jpeg')
paths_test = sorted(glob('retina-test/*.jpeg'))
len(paths), len(paths_test)

In [ ]:
idx = np.random.randint(0, len(paths), size=1)[0]
path = paths[idx]
# path = 'retina-train/5258_right_2.jpeg'
img = cv2.imread(path)# [:,:,::-1]
plt.imshow(img[:,:,::-1])
print(path)

In [ ]:
all_filenames = [os.path.split(p)[1] for p in paths] # path -> ***.jpeg
print(all_filenames[:3])
all_cls = [int(name.split('.')[0].split('_')[-1]) for name in all_filenames] # ***.jpeg -> 0~4
print(all_cls[:3])

In [ ]:
# Class distribution
cls, count = np.unique(all_cls, return_counts=True)
plt.bar(cls, count)

#### Build Transfer Learning Model

In [ ]:
# Build model

In [ ]:
model.summary()

#### Prepare Data

In [ ]:
# split dataset
path_train, path_val = train_test_split(paths, test_size=0.2)

len(path_train), len(path_val)

In [ ]:
def create_dataset(paths, img_size=IMG_SIZE, num_classes=num_classes, test=False):
    num_data = len(paths)
    x_data = np.empty((num_data, img_size, img_size, 3))
    y_data = np.empty((num_data))

    for i, path in enumerate(tqdm(paths)):
        # read image and preprocess
        img = cv2.imread(path)[:,:,::-1]
        img = cv2.resize(img, (img_size, img_size))
        # !!!!!!!!! Use model preprocessing function !!!!!!!!!!
        img = # 
        x_data[i] = img

        if not test:
            # read class label
            filename = os.path.split(path)[1]
            cls = int(filename.split('.')[0].split('_')[-1]) # '38200_left_0.jpeg' -> 0
            y_data[i] = cls
    if not test:
        y_data = utils.to_categorical(y_data, num_classes=num_classes)
        return x_data, y_data
    else:
        return x_data

In [ ]:
x_train, y_train = create_dataset(path_train)
x_val, y_val = create_dataset(path_val)
x_test = create_dataset(paths_test, test=True)

In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

#### Training

In [ ]:
model.compile(optimizer='adam', 
              loss=losses.categorical_crossentropy, 
              metrics=[metrics.categorical_accuracy])

In [ ]:
# Add data augmentation


logs = model.fit(x_train, y_train,
                batch_size=32,
                epochs=50,
                validation_data=(x_val, y_val))

#### Make Submission file

In [ ]:
predictions = np.argmax(model.predict(x_test), axis=-1)

df = pd.DataFrame()
df['Id'] = [p.split(os.sep)[-1] for p in paths_test]
df['Category'] = predictions
df.to_csv('submission.csv', index=None)
df